## Tugas 3: Diabetes Dataset with Ensemble Voting Using Logistic Regression, SVM kernel polynomial, Decission Tree

1. Import Library

In [633]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC 
from sklearn.tree import DecisionTreeClassifier 
from sklearn.ensemble import VotingClassifier 
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

2. Load Data

In [634]:
# Load data
df = pd.read_csv('data/diabetes.csv')

df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [635]:
# Cek kolom null
df.isnull().sum()

Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
Outcome                     0
dtype: int64

In [636]:
# Cek kolom neng nilai 0
feature_columns = ['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI', 'DiabetesPedigreeFunction', 'Age']
for column in feature_columns:
    print("============================================")
    print(f"{column} => Missing zeros: {len(df.loc[df[column] == 0])}")

Pregnancies => Missing zeros: 111
Glucose => Missing zeros: 5
BloodPressure => Missing zeros: 35
SkinThickness => Missing zeros: 227
Insulin => Missing zeros: 374
BMI => Missing zeros: 11
DiabetesPedigreeFunction => Missing zeros: 0
Age => Missing zeros: 0


In [637]:
# Mengubah nilai yang memiliki nilai 0 dengan mean, karena tidak masuk akal manusia yang memiliki skor insulin misal 0 total, karena seperti mustahil rasanya

# Mengubah 0 dengan Mean
from sklearn.impute import SimpleImputer

fill_values = SimpleImputer(missing_values=0, strategy="mean", copy=False)

df[feature_columns] = fill_values.fit_transform(df[feature_columns])

3. Split data

In [638]:
X = df[feature_columns]
Y = df.Outcome

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

4. Train Data dengan Voting Algorithm

In [639]:
clf1 = LogisticRegression(max_iter=1000)
clf2 = SVC(kernel='poly')
clf3 = DecisionTreeClassifier()

# Hard Voting
voting = VotingClassifier(estimators=[('LogisticRegression', clf1), ('SVM-Polynomial', clf3),('DecisionTree', clf2)], voting='hard')

voting.fit(X_train, y_train)

y_pred_vt = voting.predict(X_test)

acc_vt = accuracy_score(y_test, y_pred_vt)

print('Voting Hard')
print(f"Test set accuracy: {acc_vt*100:.2f}%")

y_pred_train_vt = voting.predict(X_train)

acc_train_vt = accuracy_score(y_train, y_pred_train_vt)

print(f"Train set accuracy: {acc_train_vt*100:.2f}%")

Voting Hard
Test set accuracy: 76.62%
Train set accuracy: 80.94%


Hasilnya masih kurang, Saya coba scale

In [640]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

feature_columns = ['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI', 'DiabetesPedigreeFunction', 'Age']
X = df[feature_columns]
y = df.Outcome

In [641]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [642]:
scaler = StandardScaler()
scaler.fit(X_train)

StandardScaler()

In [643]:
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [644]:
clf1 = LogisticRegression(max_iter=1000)
clf2 = SVC(kernel='poly')
clf3 = DecisionTreeClassifier()

# Hard Voting
voting = VotingClassifier(estimators=[('LogisticRegression', clf1), ('SVM-Polynomial', clf3),('DecisionTree', clf2)], voting='hard')

voting.fit(X_train_scaled, y_train)

y_pred_test = voting.predict(X_test_scaled)

y_pred_train = voting.predict(X_train_scaled)

acc_test = accuracy_score(y_test, y_pred_test)

acc_train = accuracy_score(y_train, y_pred_train)

print('Voting Hard (Scaled)')
print(f"Test set accuracy: {acc_test*100:.2f}%")
print(f"Train set accuracy: {acc_train*100:.2f}%")

Voting Hard (Scaled)
Test set accuracy: 77.27%
Train set accuracy: 84.36%


### Kesimpulan Perbedaan Scale atau tidak hasilnya tidak jauh beda, atau di suatu ketika ternyata sama saja